In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules
from PW_explorer.export import PWEExport
from PW_explorer.visualize import PWEVisualization

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
from copy import deepcopy
from collections import OrderedDict
import random
import csv

In [23]:
def sql2clingo(sqlCommand,originalSchema,colour='black'):
    sql = sqlCommand.replace('"','')
    sqlList = sql.split(' ') 
    #print(sqlList)
    schemaList = originalSchema.split(',')
    #print(schemaList)
    clingoString = 0
    #print(sqlList[0])
    if sqlList[0] == 'UPDATE':
        sql = sql.replace(' = ','=')
        sql = sql.replace(' > ','>')
        sql = sql.replace(' < ','<')
        sql = sql.replace(' >= ','>=')
        sql = sql.replace(' <= ','<=')
        sql = sql.replace(',','')
        sqlList = sql.split(' ') 
        setLoc = sqlList.index('SET')
        whereLoc = sqlList.index('WHERE')
        setCommands = sqlList[setLoc+1:whereLoc]
        setCommands = [x.upper() for x in setCommands]
        whereCommands = sqlList[whereLoc+1:]
        whereCommands = [x.upper() for x in whereCommands if x != 'AND']

        return [whereCommands,setCommands]
    else:
        return 0

In [13]:
testQueryNames = ['/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows10_Nqueries10.csv','/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows100_Nqueries10.csv','/home/mj1e16/abduction/pwe/jupyterNotebooks/notInPWEenv/qfixQueries_Nrows1000_Nqueries10.csv']
originalSchema = 'id,a0,a1,a2,a3,a4,a5'.upper()

In [14]:
testQueries = testQueryNames[0]
with open(testQueries,'r') as f:
    data = f.read()
#print(data)    
queries = data.split('\n')
#print(queries)

In [24]:
whereCom = []
setCom = []

for x in queries:
    clingoString = sql2clingo(x,originalSchema)
    if clingoString != 0:
        whereCom.append(clingoString[0])
        setCom.append(clingoString[1])

In [25]:
AWhere = [[],[],[],[],[]]
for x in whereCom:
    for y in x:
        if 

[['A1=3', 'A5=2'],
 ['A1>=38', 'A1<48', 'A4>=2', 'A4<12'],
 ['A1=71', 'A5=94'],
 ['A1>=53', 'A1<63', 'A4>=80', 'A4<90'],
 ['A1>=22', 'A1<32', 'A4>=72', 'A4<82'],
 ['A1=68', 'A5=37'],
 ['A1>=35', 'A1<45', 'A4>=44', 'A4<54'],
 ['A1=96', 'A5=2'],
 ['A1>=38', 'A1<48', 'A4>=2', 'A4<12'],
 ['A1=71', 'A5=94'],
 ['A1>=53', 'A1<63', 'A4>=80', 'A4<90'],
 ['A1>=22', 'A1<32', 'A4>=72', 'A4<82'],
 ['A1=68', 'A5=37'],
 ['A1>=35', 'A1<45', 'A4>=44', 'A4<54'],
 ['A4>=24', 'A4<34', 'A2>=3', 'A2<13'],
 ['A4>=68', 'A4<78', 'A2>=65', 'A2<75'],
 ['A4>=45', 'A4<55', 'A2>=75', 'A2<85'],
 ['A4>=51', 'A4<61', 'A2>=83', 'A2<93'],
 ['A0=70', 'A2=16'],
 ['A0=96', 'A2=71'],
 ['A4>=28', 'A4<38', 'A2>=18', 'A2<28'],
 ['A4>=24', 'A4<34', 'A2>=3', 'A2<13'],
 ['A1=41', 'A0=20'],
 ['A4>=68', 'A4<78', 'A2>=65', 'A2<75'],
 ['A4>=45', 'A4<55', 'A2>=75', 'A2<85'],
 ['A4>=51', 'A4<61', 'A2>=83', 'A2<93'],
 ['A0=70', 'A2=16'],
 ['A0=96', 'A2=71'],
 ['A4>=28', 'A4<38', 'A2>=18', 'A2<28'],
 ['A0>=78', 'A0<88', 'A4>=43', 'A4<53'